## 결정 트리, 랜덤 포레스트, 에이다부스트. 
#### 필요한 패키지를 불러온다:

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics, preprocessing
from scipy.stats import itemfreq

#### 데이터를 불러온다:

In [2]:
os.chdir(r"C:\Users\bit\Desktop\머신러닝 알고리즘과 응용 2\data")

In [3]:
# df = pd.read_csv('data_spam.csv', header='infer',encoding='ISO-8859-1')
df = pd.read_csv('data_spam.csv', header='infer',encoding='latin1')

In [4]:
df.shape

(1000, 63)

In [5]:
df.head(5)

,email_id,is_spam,word_freq_will,word_freq_original,word_freq_415,word_freq_mail,char_freq_#,char_freq_$,word_freq_internet,word_freq_edu,...,word_freq_receive,word_freq_000,capital_run_length_average,word_freq_address,word_freq_george,word_freq_cs,word_freq_random,word_freq_conference,word_freq_technology,char_freq_(
0,3227,no,1.06,0.26,0.0,0.00,0.0,0.00,0.00,0.0,...,0.00,0.00,2.646,0.00,0.0,0.0,0,0.0,0.00,0.039
1,2331,no,0.00,0.00,0.0,1.40,0.0,0.00,0.00,0.0,...,0.00,0.00,3.875,0.00,1.4,0.0,0,0.0,0.00,0.220
2,1899,yes,0.42,0.00,0.0,0.42,0.0,0.39,0.00,0.0,...,0.42,3.38,26.405,0.00,0.0,0.0,0,0.0,0.42,0.097
3,1387,no,0.00,0.00,0.0,0.00,0.0,0.00,2.07,0.0,...,0.00,0.00,1.523,0.00,0.0,0.0,0,0.0,0.00,0.144
4,3555,no,1.94,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.00,0.00,1.197,0.27,0.0,0.0,0,0.0,0.00,0.000


In [6]:
X=np.array(df.drop(columns='is_spam'))
Y=np.array(df.is_spam)
header = df.columns
headerX = df.drop(columns='is_spam').columns

'yes', 'no' 레이블을 숫자로 변환:

In [7]:
LE = preprocessing.LabelEncoder()
Y = LE.fit_transform(Y)

In [8]:
table = itemfreq(Y)
table

D:\Users\bit\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """Entry point for launching an IPython kernel.


array([[  0, 736],
       [  1, 264]], dtype=int64)

NaN이 있으면 채워 넣음:

In [9]:
IPT = preprocessing.Imputer()
X = IPT.fit_transform(X)

#### 데이터 전처리:

In [10]:
X = preprocessing.scale(X)

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=3)

#### Tree 적용 (교차검증 최적화 포함):

In [12]:
depth_grid = np.arange(2,31,2)
parameters = {'max_depth':depth_grid}

In [13]:
gridCV = GridSearchCV(DecisionTreeClassifier(), parameters, cv=10)
gridCV.fit(X_train, Y_train);
best_depth = gridCV.best_params_['max_depth']

In [14]:
print("Tree best depth : " + str(best_depth))

Tree best depth : 12


In [15]:
DTC_best = DecisionTreeClassifier(max_depth=best_depth)
DTC_best.fit(X_train, Y_train);
Y_pred = DTC_best.predict(X_test)
print( "Tree best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Tree best accuracy : 0.884


#### Random Forest 적용 (교차검증 최적화 포함):

Tree 하고는 스텝에서 조금 차이가 있음:

In [16]:
estimator_grid = np.arange(1, 30, 5)
depth_grid = np.arange(1, 10, 2)
parameters = {'n_estimators': estimator_grid, 'max_depth': depth_grid}
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_depth = gridCV.best_params_['max_depth']

In [17]:
print("Random Forest best n estimator : " + str(best_n_estim))
print("Random Forest best depth : " + str(best_depth))

Random Forest best n estimator : 21
Random Forest best depth : 7


In [18]:
RF_best = RandomForestClassifier(max_depth=best_depth,n_estimators=best_n_estim,random_state=3)
RF_best.fit(X_train, Y_train);
Y_pred = RF_best.predict(X_test)
print( "Random Forest best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Random Forest best accuracy : 0.928


#### AdaBoost 적용 (교차검증 최적화 포함): <br>
디폴트: base_estimator=DecisionTreeClassifier 

In [19]:
estimator_grid = np.arange(30, 80, 10)
learning_rate_grid = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
parameters = {'n_estimators': estimator_grid, 'learning_rate': learning_rate_grid}
gridCV = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_learn_rate = gridCV.best_params_['learning_rate']

In [20]:
print("Ada Boost best n estimator : " + str(best_n_estim))
print("Ada Boost best learning rate : " + str(best_learn_rate))

Ada Boost best n estimator : 40
Ada Boost best learning rate : 0.5


In [21]:
AB_best = AdaBoostClassifier(n_estimators=best_n_estim,learning_rate=best_learn_rate,random_state=3)
AB_best.fit(X_train, Y_train);
Y_pred = AB_best.predict(X_test)
print( "Ada Boost best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Ada Boost best accuracy : 0.934
